# Configuration

In [31]:
nb_users = 20
rampup_batch_size = 4
resource_capacity = 480
sim_duration = 30 #60 # 1 = 1 mn

In [32]:
#debug
#nb_users = 1
#resource_capacity = 80
#sim_duration = 180 # 1 = 1 mn

In [33]:
session_configuration = """\
session:
    name: nominal test
    scenarios:
        - name: Sc1
          weight: 10
          steps:
                loop:
                    tasks:
                        - name: process-1
                          duration: 10
                          resources : 50   
                          #repeat: 4

        - name: Sc2
          weight: 10
          steps:
                loop:
                    tasks:
                        - name: process-2
                          duration: 4
                          resources : 50   
                          #repeat: 4

"""



# Functions

In [34]:
# remove these lines if you use the pypi package
import sys
sys.path.append("../..") # go to parent dir

In [35]:
import os
import logging
import pandas as pd
import plotly.express as px

from virtualusers.world import World, Metric

In [36]:
#TODO
!pip install -U kaleido

Requirement already up-to-date: kaleido in /Users/cfalguiere/anaconda3/lib/python3.7/site-packages (0.2.1)


In [37]:
log_filename = "logs-10.log"
mainLogger = logging.getLogger()
mainLogger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
mainLogger.addHandler(fhandler)
mainLogger.setLevel(logging.DEBUG)
mainLogger.debug("test")

In [38]:
export_dir = 'export'
os.makedirs(export_dir, exist_ok=True)

## Main 

In [39]:
world = World(session_configuration, 
              nb_users = nb_users, 
              rampup_batch_size = rampup_batch_size,
              resource_capacity = resource_capacity)

In [40]:
mainLogger.setLevel(logging.INFO)
world.start(sim_duration = sim_duration) # 1 = 1mn

In [41]:
# debug
if False:
    session_configuration = """\
    session:
        name: nominal test
        scenarios:
            - name: Sc1
              weight: 2
              steps:
                    loop:
                        tasks:
                            - name: step 1 debug
                              duration: 10
                              resources : 3   
                              wait: 5
    """
    world = World(session_configuration, nb_users = 1, resource_capacity = 5 )
    mainLogger.setLevel(logging.DEBUG)
    world.start(sim_duration = 20)


# Time Series

In [42]:
df_system = world.res_agent.collect()
#display(df_system)

## Occupied slots

In [43]:
#df = dfs['occupied_slots']
df = df_system[ df_system.Metric == Metric.BS.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

In [44]:
fig.write_image(os.path.join(export_dir, "occupied_slots.png"))

## Requests waiting

In [45]:
#df = dfs['requests_waiting']
df = df_system[ df_system.Metric == Metric.RW.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

In [46]:
fig.write_image(os.path.join(export_dir, "waiting_requests.png"))

## Active Users

In [47]:
df_users_active = world.user_gen.collect()
#display(df_users_active)

In [48]:
fig = px.bar(df_users_active, x='Timestamp', y='Value')
fig.show()

In [49]:
fig.write_image(os.path.join(export_dir, "active_users.png"))

# Users activity

In [50]:
# debug
df_start = pd.DataFrame(world.user_monitor.start_data)
df_start_sorted = df_start.sort_values(by=['StartMark', 'Scenario', 'TaskId'])
display(df_start_sorted)

,Scenario,Start,StartMark,Task,TaskId,Username
1,Sc1,2021-09-14 22:29:03.581918,0,process-1,1,User-002
2,Sc1,2021-09-14 22:29:03.581918,0,process-1,1,User-003
0,Sc2,2021-09-14 22:29:03.581918,0,process-2,1,User-001
3,Sc2,2021-09-14 22:29:03.581918,0,process-2,1,User-004
4,Sc1,2021-09-14 22:30:03.581918,1,process-1,1,User-005
6,Sc1,2021-09-14 22:30:03.581918,1,process-1,1,User-007
5,Sc2,2021-09-14 22:30:03.581918,1,process-2,1,User-006
7,Sc2,2021-09-14 22:30:03.581918,1,process-2,1,User-008
8,Sc1,2021-09-14 22:31:03.581918,2,process-1,1,User-009
9,Sc1,2021-09-14 22:31:03.581918,2,process-1,1,User-010


In [51]:
# debug
#df_stop = pd.DataFrame(world.user_monitor.stop_data)
#display(df_stop)

In [52]:
# debug
#df_stop.shape

## Users chronogram

In [53]:
df_users = world.user_monitor.collect()
display(df_users)

,Scenario,Start,StartMark,Task,TaskId,Username,Finish,FinishMark,Duration
0,Sc2,2021-09-14 22:29:03.581918,0,process-2,1,User-001,2021-09-14 22:33:03.581918,4,4
1,Sc1,2021-09-14 22:29:03.581918,0,process-1,1,User-002,2021-09-14 22:39:03.581918,10,10
2,Sc1,2021-09-14 22:29:03.581918,0,process-1,1,User-003,2021-09-14 22:39:03.581918,10,10
3,Sc2,2021-09-14 22:29:03.581918,0,process-2,1,User-004,2021-09-14 22:33:03.581918,4,4
4,Sc1,2021-09-14 22:30:03.581918,1,process-1,1,User-005,2021-09-14 22:40:03.581918,11,10
5,Sc2,2021-09-14 22:30:03.581918,1,process-2,1,User-006,2021-09-14 22:34:03.581918,5,4
6,Sc1,2021-09-14 22:30:03.581918,1,process-1,1,User-007,2021-09-14 22:40:03.581918,11,10
7,Sc2,2021-09-14 22:30:03.581918,1,process-2,1,User-008,2021-09-14 22:34:03.581918,5,4
8,Sc1,2021-09-14 22:31:03.581918,2,process-1,1,User-009,2021-09-14 22:41:03.581918,12,10
9,Sc1,2021-09-14 22:31:03.581918,2,process-1,1,User-010,2021-09-14 22:43:03.581918,14,12


In [54]:
fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username", color="Scenario")
fig.update_yaxes(categoryorder='category descending')
fig.show()

In [55]:
fig.write_image(os.path.join(export_dir, "users_chronogram.png"))

## Users activity distribution

In [56]:
df_users[["Duration"]].describe()

,Duration
count,20.00000
mean,9.75000
std,4.07657
min,4.00000
25%,6.00000
50%,10.00000
75%,12.00000
max,17.00000


In [57]:
mainLogger.debug(f"max={max}")
fig = px.histogram(df_users, x="Duration", nbins=20)
fig.show()

In [58]:
fig.write_image(os.path.join(export_dir, "histo_requests_duration.png"))

In [59]:
# when starts waiting - more or less 8 users
# as much as processes - more or less 13 users

# Logs

In [60]:
with open(log_filename) as log:
            print(log.read())

2021-09-14 22:29:03,568 - root - DEBUG - test
2021-09-14 22:29:03,579 - root - INFO - creating simulation
2021-09-14 22:29:03,581 - root - INFO - session name: nominal test
2021-09-14 22:29:03,581 - root - INFO - scenarios_index: [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
2021-09-14 22:29:03,581 - root - INFO - Clock created - base 1631651343.581918
2021-09-14 22:29:03,582 - root - INFO - create resource with capacity 480
2021-09-14 22:29:03,582 - root - INFO - initial level 480
2021-09-14 22:29:03,582 - root - INFO - creating user generator for 20 users
2021-09-14 22:29:03,582 - root - INFO - creating agent
2021-09-14 22:29:03,587 - root - INFO - starting simulation
2021-09-14 22:29:03,587 - root - INFO - user created User-001
2021-09-14 22:29:03,587 - root - INFO - user created User-002
2021-09-14 22:29:03,587 - root - INFO - user created User-003
2021-09-14 22:29:03,588 - root - INFO - user created User-004
2021-09-14 22:29:03,588 - root - INFO - user created User-